In [1]:
%matplotlib inline
#import spacy
import os
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import scipy.misc
import pylab
import nltk
from scipy import optimize
from sklearn.neighbors import KDTree
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet
from nltk.stem import RegexpStemmer

# initialize wordnet lemmatizer (need to download WordNet Corpus from NLTK)
wordnet_lemmatizer = WordNetLemmatizer()
stemmer = nltk.PorterStemmer();
st = RegexpStemmer('ing$|s$|e$|able$', min=4)

stop_words = nltk.corpus.stopwords.words('english')

In [2]:
n_train = 10000
n_test  = 2000

In [3]:
resNet = []
for line in open('./data/features_train/features_resnet1000_train.csv'):
    resNet.append(line.strip().split(','))
for line in resNet:
    line[0] = int(line[0][line[0].index("/")+1:line[0].index(".")])  
resNet.sort()
for i in range(len(resNet)):
    #print resNet[i][0]
    resNet[i] = resNet[i][1:] 
m, n = len(resNet), len(resNet[0])
vis = np.zeros((m,n), dtype=np.float32)
for i in range(m):
    for j in range(n):
        vis[i,j] = float(resNet[i][j])

In [4]:
# build dictionary based-on 2nd discriptions
dictionary = []
sentenceCounter = defaultdict(int)

for i in range(n_train):  
    for line in open('./data/descriptions_train/' + str(i) + '.txt'):
        for w in line.strip().split('.')[0].split():
            w = st.stem(w.lower())
            if w not in stop_words:
                w = wordnet_lemmatizer.lemmatize(w)
                if len(wordnet.synsets(w)) != 0:
                    if str(wordnet.synsets(w)[0])[str(wordnet.synsets(w)[0]).index(".")+1:str(wordnet.synsets(w)[0]).index(".")+2] == 'n':
                        sentenceCounter[w] += 1  
""""""                    
for i in range(n_test):  
    for line in open('./data/descriptions_test/' + str(i) + '.txt'):
        for w in line.strip().split('.')[0].split():
            w = st.stem(w.lower())
            if w not in stop_words:
                w = wordnet_lemmatizer.lemmatize(w)
                if len(wordnet.synsets(w)) != 0:
                    if str(wordnet.synsets(w)[0])[str(wordnet.synsets(w)[0]).index(".")+1:str(wordnet.synsets(w)[0]).index(".")+2] == 'n':
                        sentenceCounter[w] += 1   
""""""   
for k,v in sentenceCounter.items():
        #print k, v
        dictionary.append(k)         

In [5]:
# dictionary = []

# binary BoW representation of test tags 
# tag_bow = np.zeros((n_train,len(dictionary)), dtype=np.float32)
# for i in range(2000):
#     for line in open('./data/tags_train/' + str(i) + '.txt'):
#         w = line.strip().split(':')[1]
#         if w in dictionary:
#             tag_bow[i, dictionary.index(w)] = 1.0

# binary BoW representation of train descriptions
description_bow = np.zeros((n_train, len(dictionary)), dtype=np.float32)
for i in range(n_train):
    for line in open('./data/descriptions_train/' + str(i) + '.txt'):
        for w in line.strip().split('.')[0].split():
            w = st.stem(w.lower())
            if w not in stop_words:
                w = wordnet_lemmatizer.lemmatize(w)
                if w in dictionary:
                    description_bow[i, dictionary.index(w)] = 1.0


In [6]:
Y = vis
X = description_bow

In [7]:
# binary BoW representation of test descriptions
test_bow = np.zeros((n_test,len(dictionary)), dtype=np.float32)
for i in range(n_test):
    for line in open('./data/descriptions_test/' + str(i) + '.txt'):
        for w in line.strip().split('.')[0].split():
            w = st.stem(w.lower())
            if w not in stop_words:
                w = wordnet_lemmatizer.lemmatize(w)
                if w in dictionary:
                    test_bow[i, dictionary.index(w)] = 1.0

test_resNet = []
for line in open('./data/features_test/features_resnet1000_test.csv'):
    test_resNet.append(line.strip().split(','))
for line in test_resNet:
    line[0] = int(line[0][line[0].index("/")+1:line[0].index(".")])  
test_resNet.sort()
for i in range(len(test_resNet)):
    #print resNet[i][0]
    test_resNet[i] = test_resNet[i][1:]

In [39]:
total_bow = np.vstack((description_bow, test_bow))
total_bow.shape
total_bow_c = total_bow

In [9]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1750)
total_bow = pca.fit_transform(total_bow)

In [10]:
from sklearn.neural_network import MLPRegressor

In [32]:
train_x = total_bow[0:10000] # 这里是training的description
train_y = vis # 这里是train的resNet
# 在这里加入模型
reg = MLPRegressor(solver='sgd', activation='logistic', shuffle=True)
reg.fit(train_x[0:10000], train_y[0:10000])

MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [54]:
reg2 = MLPRegressor(solver='sgd', activation='logistic', shuffle=True)
reg2.fit(train_y[0:10000], total_bow_c[0:10000])

MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [57]:
print reg2.predict(train_y)[0][0:100]
dictionary[68]

[ -7.80048998e-04  -4.18119035e-03   3.68575079e-02   1.35388812e-02
  -3.82191851e-03   1.32645491e-02   7.45818794e-03   3.61580916e-03
  -6.92983609e-04  -2.45474164e-03   2.01861653e-03   5.08073234e-04
  -6.59286851e-04   5.18716992e-04   2.93124724e-03   5.09113906e-03
  -8.28109997e-04   3.00667158e-03  -2.35562185e-03   2.65139284e-03
  -2.89044739e-04  -4.13516753e-03   2.70556877e-03   2.91133016e-03
   1.96478974e-03   4.49801127e-03  -3.68479945e-03   7.03438924e-03
   3.58302552e-03  -2.65235902e-03   1.17216739e-03   2.59737328e-05
   4.27019155e-03  -1.03495225e-03   5.04334082e-03  -2.91027763e-04
   7.74750826e-04   5.41330428e-03  -2.48178202e-03   2.78384296e-03
  -1.99723469e-03   6.53394791e-03   1.20223718e-03   5.21052205e-03
  -3.59593081e-03  -5.30633531e-03  -2.88893505e-03  -3.23816418e-03
   4.17292186e-03  -1.04144284e-03  -1.31193224e-03   3.09554642e-03
   2.27043647e-03   2.67660176e-03  -2.71027829e-03   4.32924510e-03
  -1.94500510e-04   3.01641721e-03

'street'

In [60]:
kdt = KDTree(test_resNet)
kNumber = 20
dist, ind = kdt.query(reg.predict(total_bow[10000:]), k=kNumber)
# output rank list on test set
f = open('bow_baseline.csv', 'w')
f.write('Descritpion_ID,Top_20_Image_IDs\n')
for i in range(2000):
    f.write(str(i)+'.txt,')
    for j in range(kNumber):
        if j == kNumber-1:
            f.write(str(ind[i,j])+'.jpg\n')
        else:
            f.write(str(ind[i,j])+'.jpg ')
f.close()

ValueError: shapes (2000,4125) and (1750,100) not aligned: 4125 (dim 1) != 1750 (dim 0)

In [ ]:
kdt = KDTree()

In [58]:
my_data = np.loadtxt('./bow_baseline.csv', delimiter=' ', skiprows=1, dtype='str')
counter = 0
res = 0
included = []
for i in range(2000):
    my_data[i][0] = my_data[i][0].split(',')[1]
    correctFileName = str(counter)+'.jpg'
    if correctFileName in my_data[i]:
        #res += 1
        res += (20 + 1 - np.where(my_data[i] == correctFileName)[0][0]) / 20.0
        included.append([i, np.where(my_data[i] == correctFileName)])
    counter += 1

print(res) # 这个显示每条数据的的排序结果
print("Score = " + str(res/float(2000)))

9.25
Score = 0.004625
